In [1]:
# imports
import numpy as np
import pandas as pd
import statsmodels.api as sm

Build a regression model.

In [2]:
CombinedPointData = pd.read_csv('../data/CombinedPointData.csv')
CombinedPointData

,Foursquare Name,Foursquare Popularity,Foursquare Price,Matched Name,Yelp Name,Yelp Price,Search,Category,Rounded Latitude,Rounded Longitude,Rating,Free Bikes
0,Gula Villan,0.305123,Expensive,Gula Villan,Gula Villan,Affordable,13032,"Cafe, Coffee, And Tea House",60.141,24.757,8.200000,0.0
1,Iso-Vasikkasaaren kahvila,0.000000,Not Listed,NaN,NaN,NaN,13034,Café,60.141,24.757,7.139481,0.0
2,Villa Pentry,0.982601,Affordable,Villa Pentry,Villa Pentry,Not Listed,13336,Scandinavian Restaurant,60.145,24.736,8.850000,0.0
3,Grande Buffet,0.949260,Not Listed,NaN,NaN,NaN,13030,Buffet,60.145,24.914,7.139481,0.0
4,Toten seinä,0.885058,Affordable,NaN,NaN,NaN,13073,Australian Restaurant,60.146,24.752,7.139481,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6087,HSL V8002 Oskarintie,0.907233,Not Listed,NaN,NaN,NaN,19043,Bus Stop,60.335,25.077,7.139481,0.0
6088,HSL V8001 Oskarintie,0.195755,Not Listed,NaN,NaN,NaN,19043,Bus Stop,60.335,25.077,7.139481,0.0
6089,HSL V7213 Rekolanmäen koulu,0.096578,Not Listed,NaN,NaN,NaN,19043,Bus Stop,60.336,25.057,7.139481,0.0
6090,Rekolan koirapuisto,0.934974,Not Listed,Uutelan koirapuisto,NaN,NaN,16033,Dog Park,60.337,25.076,7.139481,0.0


In [3]:
# X are features Y is target.
X = CombinedPointData[['Rounded Latitude','Rounded Longitude','Rating']]
y = CombinedPointData['Free Bikes']

In [4]:
X

,Rounded Latitude,Rounded Longitude,Rating
0,60.141,24.757,8.200000
1,60.141,24.757,7.139481
2,60.145,24.736,8.850000
3,60.145,24.914,7.139481
4,60.146,24.752,7.139481
...,...,...,...
6087,60.335,25.077,7.139481
6088,60.335,25.077,7.139481
6089,60.336,25.057,7.139481
6090,60.337,25.076,7.139481


In [5]:
y

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
6087    0.0
6088    0.0
6089    0.0
6090    0.0
6091    0.0
Name: Free Bikes, Length: 6092, dtype: float64

In [6]:
# checking for correlation. the closwer a value is to 1, the more correlated; no codependency issues here.
CorrelationMatrix = CombinedPointData.corr(numeric_only=True)
CorrelationMatrix

,Foursquare Popularity,Rounded Latitude,Rounded Longitude,Rating,Free Bikes
Foursquare Popularity,1.000000,-0.136776,-0.016235,-0.044245,0.033232
Rounded Latitude,-0.136776,1.000000,0.287681,-0.135359,-0.008736
Rounded Longitude,-0.016235,0.287681,1.000000,-0.004864,0.004089
Rating,-0.044245,-0.135359,-0.004864,1.000000,-0.003786
Free Bikes,0.033232,-0.008736,0.004089,-0.003786,1.000000


In [7]:
X = sm.add_constant(X) # adding a constant
LinReg = sm.OLS(y,X)

In [8]:
Model = LinReg.fit()

Provide model output and an interpretation of the results. 

In [9]:
ModelOutput = Model.summary()
print(ModelOutput)

                            OLS Regression Results                            
Dep. Variable:             Free Bikes   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.3076
Date:                Fri, 09 Aug 2024   Prob (F-statistic):              0.820
Time:                        14:01:28   Log-Likelihood:                -17386.
No. Observations:                6092   AIC:                         3.478e+04
Df Residuals:                    6088   BIC:                         3.481e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                57.4879     72.04

The adjusted R-squared shows us that the model explains 0% of the data.

The p-values are all above 0.05 threshold. This means that the relationship between each feature (Rounded Latitude, Rounded Longitude, and Rating) and the number of free bikes is most likely from natural variation as opposed to the number of free bikes being related to the feature.

As explained by the R-squared and p-values, the features likely don't actually have any effect on the target. So although the model's coefficient values show that Rounded Latitude and Rating have a weak negative impact on bikes (when one goes up the other goes down, vice versa) and Rounded Longitude has a weak positive impact on bikes, this doesn't indicate an actual change as calculated by the model.

In [10]:
# adjusting the model by removing the feature with the highest p-value
xAdjusted = X.drop(columns=['Rating'])
xAdjusted = sm.add_constant(xAdjusted) # adding a constant
LinRegAdj = sm.OLS(y,xAdjusted)
ModelAdjusted = LinRegAdj.fit()
ModelAdjustedOutput = ModelAdjusted.summary()
print(ModelAdjustedOutput)

                            OLS Regression Results                            
Dep. Variable:             Free Bikes   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.3771
Date:                Fri, 09 Aug 2024   Prob (F-statistic):              0.686
Time:                        14:01:46   Log-Likelihood:                -17386.
No. Observations:                6092   AIC:                         3.478e+04
Df Residuals:                    6089   BIC:                         3.480e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                53.2250     71.29

In [11]:
# p values still above 0.05 threshold so adjusting again
xReadjusted = xAdjusted.drop(columns=['Rounded Latitude'])
xReadjusted = sm.add_constant(xReadjusted) # adding a constant
LinRegReadj = sm.OLS(y,xReadjusted)
ModelReadjusted = LinRegReadj.fit()
ModelReadjustedOutput = ModelReadjusted.summary()
print(ModelReadjustedOutput)

                            OLS Regression Results                            
Dep. Variable:             Free Bikes   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1018
Date:                Fri, 09 Aug 2024   Prob (F-statistic):              0.750
Time:                        14:01:48   Log-Likelihood:                -17387.
No. Observations:                6092   AIC:                         3.478e+04
Df Residuals:                    6090   BIC:                         3.479e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -3.3032     13.55

Adjusting the model doesn't have the model explain any more of the data.

# Stretch

How can you turn the regression model into a classification model?

I would get the 'Category' and 'Price' columns ready by making the values in them into separate columns and having 1 mean that data point has that value and 0 mean that row does not have that value. Using price as an example, there are 5 possible outcomes ['Not Listed', 'Most Affordable', 'Affordable', 'Expensive', 'Most Expensive']. 

I would make 'Not Listed', 'Most Affordable', 'Affordable', 'Expensive', and 'Most Expensive' columns. A data point that has a Price value of 'Not Listed' would have 1 in 'Not Listed' column and 0 for the other 4 columns made from the Price values.

This could be applied in the same way to categories. I would further clean up and group the categories, then take the final categories and make columns. If I had a data point that was a restaurant, for example, it would be a 1 in the 'Restaurant' column and 0 in all other Category related columns.